# Validación metodología ANID

El objetivo de este notebook es validar cuantitativa y cualitativamente la metodología empleada para clasificar los proyectos ANID, donde se empleó el modelo de lenguaje GPT 4 para clasificar cada proyecto. Para ello, se hará una etiquetarán manualmente algunos proyectos, y se analizará la concordancia entre las etiquetas manuales y aquellas provistas por GPT.

In [1]:
import os
import math
import requests
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import itertools
from google.colab import drive
from sklearn.metrics import classification_report

drive.mount('/content/drive')

Mounted at /content/drive


## Abrir datos

In [20]:
data_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT4_Análisis temática definida/Proyectos/ANID/Datos"
results_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT4_Análisis temática definida/Proyectos/ANID/Resultados"
df_anid = pd.read_excel(os.path.join(data_dir, "BDH_HISTORICA.xlsx"))
df_anid

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,GRUPO_DE_EVALUACION,DURACION_MESES,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO
0,ECOS190041,35152,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,MOVILIDAD - ECOS,PROGRAMA DE COOPERACION CIENTIFICA ECOS,2019,2020,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,CIENCIAS SOCIALES,...,NO APLICA,36.0,PERSONA NATURAL,JIMENA TORRES,MUJER,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,9280,NaN
1,FP200006,35312,"REDES, ESTRATEGIA Y CONOCIMIENTO",INFORMACION CIENTIFICA,FPRC,FONDO DE PUBLICACION DE REVISTAS CIENTIFICAS 2020,2020,2020,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,SIN INFORMACION,...,NO APLICA,24.0,PERSONA JURIDICA,FABIANA MARTIN,MUJER,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,12005.582,NaN
2,FOVI210059,37149,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,CIENCIAS NATURALES,...,NO APLICA,12.0,PERSONA JURIDICA,LAURA SANCHEZ-JARDON,MUJER,FUNDACION OMORA ONG,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,20000,NaN
3,FOVI210061,37151,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,CIENCIAS NATURALES,...,NO APLICA,12.0,PERSONA JURIDICA,SERGIO RADIC,HOMBRE,UNIVERSIDAD DE MAGALLANES,AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,20000,NaN
4,ED230064,34491,NO APLICA,EXPLORA,PROYECTOS DE DIVULGACION Y VALORACION DE LA CI...,XXIII CONCURSO NACIONAL DE PROYECTOS EXPLORA P...,2019,2019,PODCAST ANTARTICA,MULTIDISCIPLINARIO,...,NO APLICA,12.0,PERSONA NATURAL,PAULINA ROJAS,MUJER,INSTITUTO ANTARTICO CHILENO (INACH),AUSTRAL,12. MAGALLANES Y ANTARTICA CHILENA,22000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42073,CL-2023B-012,42074,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,RAPID RESPONSE TOO SPECTROSCOPIC OBSERVATIONS ...,CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"PIGNATA, GIULIANO",HOMBRE,UNIVERSIDAD NACIONAL ANDRÉS BELLO,RM,13. METROPOLITANA,0,NO ENTREGA RECURSOS
42074,CL-2023B-015,42075,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,PROBING CLOUDS AND HAZES IN AN INFLATED WARM-J...,CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"RAJKUMAR, ANITHA RAJ",HOMBRE,UNIVERSIDAD DE ATACAMA,NORTE,03. ATACAMA,0,NO ENTREGA RECURSOS
42075,CL-2023B-016,42076,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,PROBING THE FORMATION HISTORIES OF NSCS IN THE...,CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"NAYAK, PRASANTA KUMAR",MUJER,PONTIFICIA UNIVERSIDAD CATÓLICA DE CHILE,RM,13. METROPOLITANA,0,NO ENTREGA RECURSOS
42076,CL-2023B-017,42077,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,"ELUCIDATING THE NATURE OF GRB\,220831A'S OPTIC...",CIENCIAS NATURALES,...,NO APLICA,5.0,PERSONA NATURAL,"TEJOS, NICOLAS",HOMBRE,PONTIFICIA UNIVERSIDAD CATÓLICA DE VALPARAÍSO,CENTRO,05. VALPARAISO,0,NO ENTREGA RECURSOS


In [21]:
df_class = pd.read_csv(os.path.join(results_dir, "clasificaciones_anid_enfermedades_especificas_limpia.csv"))
df_class


,código,glosa,clasificación,clasificación_normalizada
0,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro,otro
1,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro,otro
2,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro,otro
3,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro,otro
4,ED230064,PODCAST ANTARTICA,otro,otro
...,...,...,...,...
41393,31180026,COSMOS IN THE KINDER: EMPOWERING SCIENTIFIC TH...,otro,otro
41394,31180032,DEVELOPING ASTRONOMY TOOLS FOR BLIND AND VISUA...,otro.,otro
41395,31180035,STAR FORMATION THROUGH TIME: FROM NEAR TO FAR,otro.,otro
41396,31180046,ANNUAL MEETING SOCHIAS 2020,otro.,otro


In [22]:
df_health = pd.read_csv(os.path.join(results_dir, "clasificaciones_anid_salud_otro_limpia.csv"))
df_health


,código,glosa,clasificación,clasificación_normalizada
0,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro,otro
1,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro,otro
2,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro,otro
3,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro,otro
4,ED230064,PODCAST ANTARTICA,otro,otro
...,...,...,...,...
21001,FOVI210067,FORTALECIENDO LA PARTICIPACION CIUDADANA EN EL...,otro,otro
21002,SUC210102,PNX3: UNA REVOLUCION EN EL TRATAMIENTO DEL DOL...,otro.,otro
21003,SUC210108,VALIDACION DE UN ADITIVO DIETARIO FUNCIONAL PA...,otro.,otro
21004,FOVI210064,FORTALECIMIENTO DE LAS CAPACIDADES DE FORMACIO...,otro.,otro


## Cruzar datos

In [24]:
df_anid["id"] = df_anid["CODIGO_PROYECTO"].astype(str) + " " + df_anid["NOMBRE_PROYECTO"].astype(str)
df_class["id"] = df_class["código"] + " " + df_class["glosa"]
df_health["id"] = df_health["código"] + " " + df_health["glosa"]

In [25]:
df = df_anid.merge(df_class, left_on="id", right_on="id").merge(df_health, left_on="id", right_on="id", suffixes=["_específica", "_salud"])
df

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,NOTA_MONTO,id,código_específica,glosa_específica,clasificación_específica,clasificación_normalizada_específica,código_salud,glosa_salud,clasificación_salud,clasificación_normalizada_salud
0,ECOS190041,35152,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,MOVILIDAD - ECOS,PROGRAMA DE COOPERACION CIENTIFICA ECOS,2019,2020,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,CIENCIAS SOCIALES,...,NaN,ECOS190041 ARQUEOLOGIA Y AMBIENTE: INTERACCION...,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro,otro,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro,otro
1,FP200006,35312,"REDES, ESTRATEGIA Y CONOCIMIENTO",INFORMACION CIENTIFICA,FPRC,FONDO DE PUBLICACION DE REVISTAS CIENTIFICAS 2020,2020,2020,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,SIN INFORMACION,...,NaN,FP200006 IMPLEMENTACION DE FLUJO CONTINUO DE L...,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro,otro,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro,otro
2,FOVI210059,37149,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,CIENCIAS NATURALES,...,NaN,FOVI210059 LIQUENES SUBANTARTICOS UN NUEVO CEN...,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro,otro,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro,otro
3,FOVI210061,37151,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,CIENCIAS NATURALES,...,NaN,FOVI210061 CONSOLIDACION DE LA RED BINACIONAL ...,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro,otro,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro,otro
4,ED230064,34491,NO APLICA,EXPLORA,PROYECTOS DE DIVULGACION Y VALORACION DE LA CI...,XXIII CONCURSO NACIONAL DE PROYECTOS EXPLORA P...,2019,2019,PODCAST ANTARTICA,MULTIDISCIPLINARIO,...,NaN,ED230064 PODCAST ANTARTICA,ED230064,PODCAST ANTARTICA,otro,otro,ED230064,PODCAST ANTARTICA,otro,otro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21020,CL-2023B-012,42074,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,RAPID RESPONSE TOO SPECTROSCOPIC OBSERVATIONS ...,CIENCIAS NATURALES,...,NO ENTREGA RECURSOS,CL-2023B-012 RAPID RESPONSE TOO SPECTROSCOPIC ...,CL-2023B-012,RAPID RESPONSE TOO SPECTROSCOPIC OBSERVATIONS ...,otro,otro,CL-2023B-012,RAPID RESPONSE TOO SPECTROSCOPIC OBSERVATIONS ...,otro,otro
21021,CL-2023B-015,42075,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,PROBING CLOUDS AND HAZES IN AN INFLATED WARM-J...,CIENCIAS NATURALES,...,NO ENTREGA RECURSOS,CL-2023B-015 PROBING CLOUDS AND HAZES IN AN IN...,CL-2023B-015,PROBING CLOUDS AND HAZES IN AN INFLATED WARM-J...,otro,otro,CL-2023B-015,PROBING CLOUDS AND HAZES IN AN INFLATED WARM-J...,otro,otro
21022,CL-2023B-016,42076,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,PROBING THE FORMATION HISTORIES OF NSCS IN THE...,CIENCIAS NATURALES,...,NO ENTREGA RECURSOS,CL-2023B-016 PROBING THE FORMATION HISTORIES O...,CL-2023B-016,PROBING THE FORMATION HISTORIES OF NSCS IN THE...,otro,otro,CL-2023B-016,PROBING THE FORMATION HISTORIES OF NSCS IN THE...,otro,otro
21023,CL-2023B-017,42077,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,"ELUCIDATING THE NATURE OF GRB\,220831A'S OPTIC...",CIENCIAS NATURALES,...,NO ENTREGA RECURSOS,"CL-2023B-017 ELUCIDATING THE NATURE OF GRB\,22...",CL-2023B-017,"ELUCIDATING TH

In [9]:
df_valid = df[df["AGNO_FALLO"] >= 2013]
df_valid

,CODIGO_PROYECTO,N,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,NOTA_MONTO,id,código_específica,glosa_específica,clasificación_específica,clasificación_normalizada_específica,código_salud,glosa_salud,clasificación_salud,clasificación_normalizada_salud
0,ECOS190041,35152,"REDES, ESTRATEGIA Y CONOCIMIENTO",PCI,MOVILIDAD - ECOS,PROGRAMA DE COOPERACION CIENTIFICA ECOS,2019,2020,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,CIENCIAS SOCIALES,...,NaN,ECOS190041 ARQUEOLOGIA Y AMBIENTE: INTERACCION...,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro,otro,ECOS190041,ARQUEOLOGIA Y AMBIENTE: INTERACCION CULTURAL E...,otro,otro
1,FP200006,35312,"REDES, ESTRATEGIA Y CONOCIMIENTO",INFORMACION CIENTIFICA,FPRC,FONDO DE PUBLICACION DE REVISTAS CIENTIFICAS 2020,2020,2020,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,SIN INFORMACION,...,NaN,FP200006 IMPLEMENTACION DE FLUJO CONTINUO DE L...,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro,otro,FP200006,IMPLEMENTACION DE FLUJO CONTINUO DE LAS REVIST...,otro,otro
2,FOVI210059,37149,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,CIENCIAS NATURALES,...,NaN,FOVI210059 LIQUENES SUBANTARTICOS UN NUEVO CEN...,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro,otro,FOVI210059,LIQUENES SUBANTARTICOS UN NUEVO CENTINELA DEL ...,otro,otro
3,FOVI210061,37151,"REDES, ESTRATEGIA Y CONOCIMIENTO",REC,FOMENTO VINCULACION INTERNACIONAL - FOVI,FOMENTO VINCULACION INTERNACIONAL PARA INSTITU...,2021,2021,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,CIENCIAS NATURALES,...,NaN,FOVI210061 CONSOLIDACION DE LA RED BINACIONAL ...,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro,otro,FOVI210061,CONSOLIDACION DE LA RED BINACIONAL (CHILE-ARGE...,otro,otro
4,ED230064,34491,NO APLICA,EXPLORA,PROYECTOS DE DIVULGACION Y VALORACION DE LA CI...,XXIII CONCURSO NACIONAL DE PROYECTOS EXPLORA P...,2019,2019,PODCAST ANTARTICA,MULTIDISCIPLINARIO,...,NaN,ED230064 PODCAST ANTARTICA,ED230064,PODCAST ANTARTICA,otro,otro,ED230064,PODCAST ANTARTICA,otro,otro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21020,CL-2023B-012,42074,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,RAPID RESPONSE TOO SPECTROSCOPIC OBSERVATIONS ...,CIENCIAS NATURALES,...,NO ENTREGA RECURSOS,CL-2023B-012 RAPID RESPONSE TOO SPECTROSCOPIC ...,CL-2023B-012,RAPID RESPONSE TOO SPECTROSCOPIC OBSERVATIONS ...,otro,otro,CL-2023B-012,RAPID RESPONSE TOO SPECTROSCOPIC OBSERVATIONS ...,otro,otro
21021,CL-2023B-015,42075,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,PROBING CLOUDS AND HAZES IN AN INFLATED WARM-J...,CIENCIAS NATURALES,...,NO ENTREGA RECURSOS,CL-2023B-015 PROBING CLOUDS AND HAZES IN AN IN...,CL-2023B-015,PROBING CLOUDS AND HAZES IN AN INFLATED WARM-J...,otro,otro,CL-2023B-015,PROBING CLOUDS AND HAZES IN AN INFLATED WARM-J...,otro,otro
21022,CL-2023B-016,42076,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,PROBING THE FORMATION HISTORIES OF NSCS IN THE...,CIENCIAS NATURALES,...,NO ENTREGA RECURSOS,CL-2023B-016 PROBING THE FORMATION HISTORIES O...,CL-2023B-016,PROBING THE FORMATION HISTORIES OF NSCS IN THE...,otro,otro,CL-2023B-016,PROBING THE FORMATION HISTORIES OF NSCS IN THE...,otro,otro
21023,CL-2023B-017,42077,"REDES, ESTRATEGIA Y CONOCIMIENTO",ASTRONOMIA,TIEMPO DE TELESCOPIO GEMINI SUR,TIEMPO DE TELESCOPIO GEMINI SUR 2023 - B,2023,2023,"ELUCIDATING THE NATURE OF GRB\,220831A'S OPTIC...",CIENCIAS NATURALES,...,NO ENTREGA RECURSOS,"CL-2023B-017 ELUCIDATING THE NATURE OF GRB\,22...",CL-2023B-017,"ELUCIDATING TH

## Validación

### Discrepancias entre ambas clasificaciones

Revisaremos aquellos casos donde para la clasificación salud vs otro quedó con la etiqueta `otro`, pero en la clasificación de enfermedades específicas sí quedó con alguna enfermedad (`cáncer`, `diabetes` o `cardiaca`)

In [10]:
with pd.option_context('display.max_colwidth', 1000):
  display(df_valid.loc[(df_valid["clasificación_normalizada_salud"] == "otro") &
                       (df_valid["clasificación_normalizada_específica"] != "otro"),
                       ["glosa_específica", "clasificación_normalizada_salud", "clasificación_normalizada_específica"]])

,glosa_específica,clasificación_normalizada_salud,clasificación_normalizada_específica
2911,ESTUDIO TEORICO DE INTERACCION DE COMPLEJOS HEXANUCLEARES DE RENIO CON EL ADN,otro,cáncer
14669,"MUSICA, CUERPO Y AFECTIVIDAD: UN RELATO MUSICOLOGICO SOBRE REPRESENTACION DE MASCULINIDADES EN CANCIONES POPULARES CHILENAS (1960-2014)",otro,cáncer
18041,DEVELOPMENT AND APPLICATION OF THE DIABATIC MODEL OF INTERMEDIATE STABILIZATION FOR REACTION MECHANISM ANALYSIS AND CATALYST DESIGN,otro,diabetes
18544,IMMOBILIZED BIOCATALYST ENGINEERING OF GLUCOSE OXIDASE AND GALACTOSE OXIDASE TOWARD ENZYMATIC TOOLBOX GENERATION FOR SUGAR ACID PRODUCTION,otro,diabetes
20999,SWEET PEP+: INGREDIENTE FUNCIONAL SUSTITUTO DE EDULCORANTES Y AZÚCAR,otro,diabetes


Son sólo 5 casos.

### Validación por muestreo aleatorio

Seleccionaremos algunos casos al azar, los etiquetaremos manualmente y luego compararemos los resultados con la clasificación entregada por GPT 4.

Para realizar esto, se iteró varias veces hasta encontrar un conjunto de 30 muestras que satisficieran la condición de tener al menos una muestra clasificada como cada una de las enfermedades específicas. Al encontrar un conjunto que cumplía dicha condición, se fijó ese conjunto (los índices del dataframe) y se procedió a etiquetar manualmente las muestras.

In [70]:
indexes = [2170, 14079, 7101, 19195, 12862, 9721, 2079, 18238, 9659, 12896, 5382, 14552, 4885, 20805, 3442, 6673, 18852, 19043, 13649, 20178, 3045, 8194, 2966, 6327, 5526, 11436, 4486, 8680, 2366, 5990]

In [71]:
df_sample = df_valid.loc[indexes][["glosa_específica", "clasificación_normalizada_salud", "clasificación_normalizada_específica"]].copy().reset_index()
with pd.option_context('display.max_colwidth', 500):
  display(df_sample[["glosa_específica"]])

,glosa_específica
0,SISTEMA INTEGRADO DE OBSERVACION Y ANALISIS DEL COMPORTAMIENTO HUMANO
1,CONVERAY: DESARROLLO Y OPTIMIZACION DE UN DISPOSITIVO DE HAZ CONVERGENTE ADAPTABLE A UN LINAC PARA USO EN RADIOTERAPIA Y RADIOCIRUGIA
2,CREACION DE UNA UNIDAD DE GESTION Y DIFUSION DE REVISTAS CIENTIFICAS UFRO: INCREMENTO DE INDEXACION Y POSICIONAMIENTO INTERNACIONAL
3,DESARROLLO ESTRATEGICO DEL AREA DE MATERIA CONDENSADA EN EL DEPARTAMENTO DE FISICA DE LA UNIVERSIDAD DE SANTIAGO DE CHILE.
4,INTERRELATION BETWEEN THE HYDRAULIC STRATEGY AND POST-DROUGHT RECOVERY CAPACITY OF THE HYDRAULIC FUNCTION AND GAS EXCHANGE AS BIOPHYSICAL DETERMINANTS OF RESILIENCE OF SIX FRUIT TREES SPECIES CULTIVATED IN ARID AND SEMI-ARID ZONE OF NORTHERN CHILE.
5,ESTUDIO DEL GEN DSX DE LOBESIA BOTRANA COMO POSIBLE HERRAMIENTA PARA LA GENERACION DE UN SISTEMA DE EXPRESION HEMBRA-ESPECIFICO APLICABLE AL CONTROL DE LA PLAGA
6,DESARROLLO DE HABILIDADES TIC PARA EL APRENDIZAJE A TRAVES DE ACTIVIDADES DE APRENDIZAJE MEDIADAS PO R ASISTENTES DIGITALES
7,"OXIDIZED VITAMIN C, AN ACTIVATOR OF MOTOR NEURON NEURITES DEGENERATION AND CELL DEATH IN AMYOTROPHIC LATERAL SCLEROSIS"
8,MATER CHILENSIS: MATERIALITY AND THE MATERNAL TURN IN CHILEAN CONTEMPORARY ART (1970S-PRESENT)
9,"MICROPOLITICS OF CARE: LOGICS, PRACTICES, AND ENGAGEMENTS RELATED TO MENTAL HEALTH DIAGNOSES AND SPECIAL EDUCATIONAL NEEDS IN CHILEAN SCHOOLS"


Anotamos las respuestas que nosotros daríamos para la clasificación salud vs otro.

In [73]:
health_answers = ["otro", "salud", "otro", "otro", "otro",
                  "otro", "otro", "salud", "otro", "salud",
                  "otro", "otro", "otro", "salud", "salud",
                  "otro", "salud", "otro", "otro", "otro",
                  "salud", "otro", "otro", "salud", "otro",
                  "otro", "otro", "otro", "otro", "otro"]
df_sample["respuestas_salud"] = health_answers

Anotamos ahora as respuestas que nosotros daríamos para la clasificación de enfermedades específicas

In [74]:
diseases_answers = ["otro","cáncer","otro","otro","otro",
                    "otro","otro","otro","otro","otro",
                    "otro","otro","otro","otro","otro",
                    "otro","otro","otro","otro","otro",
                    "cáncer","otro","otro","otro","otro",
                    "otro","otro","otro","otro","otro"]

df_sample["respuestas_específicas"] = diseases_answers

Veamos ahora los resultados

In [75]:
df_sample

,index,glosa_específica,clasificación_normalizada_salud,clasificación_normalizada_específica,respuestas_salud,respuestas_específicas
0,2170,SISTEMA INTEGRADO DE OBSERVACION Y ANALISIS DE...,otro,otro,otro,otro
1,14079,CONVERAY: DESARROLLO Y OPTIMIZACION DE UN DISP...,salud,cáncer,salud,cáncer
2,7101,CREACION DE UNA UNIDAD DE GESTION Y DIFUSION D...,otro,otro,otro,otro
3,19195,DESARROLLO ESTRATEGICO DEL AREA DE MATERIA CON...,otro,otro,otro,otro
4,12862,INTERRELATION BETWEEN THE HYDRAULIC STRATEGY A...,otro,otro,otro,otro
5,9721,ESTUDIO DEL GEN DSX DE LOBESIA BOTRANA COMO PO...,otro,otro,otro,otro
6,2079,DESARROLLO DE HABILIDADES TIC PARA EL APRENDIZ...,otro,otro,otro,otro
7,18238,"OXIDIZED VITAMIN C, AN ACTIVATOR OF MOTOR NEUR...",salud,otro,salud,otro
8,9659,MATER CHILENSIS: MATERIALITY AND THE MATERNAL ...,otro,otro,otro,otro
9,12896,"MICROPOLITICS OF CARE: LOGICS, PRACTICES, AND ...",salud,otro,salud,otro


#### Salud vs otro

A simple vista, parece bien. Veamos los casos donde hay discrepancias para la clasificación salud vs otro.

In [78]:
with pd.option_context('display.max_colwidth', 500):
  display(df_sample.loc[df_sample["clasificación_normalizada_salud"] != df_sample["respuestas_salud"], ["glosa_específica", "clasificación_normalizada_salud", "respuestas_salud"]])

,glosa_específica,clasificación_normalizada_salud,respuestas_salud
18,"THIOCOMPOSITES, A NOVEL DRUG CARRIER SYSTEM: PROOF-OF-CONCEPT FOR A POTENTIAL APPLICATION IN AQUACULTURE",salud,otro
26,TRANSMISSION DYNAMICS OF LEPTOSPIRA WITHIN-FARM: UNDERSTANDING AND MODELLING THE EFFECTS OF SOME HOSTS ON LEPTOSPIRA DYNAMICS AND MULTIPLE HOST-MULTIPLE STRAIN INTERACTIONS.,salud,otro
28,MODELING AND OPTIMIZATION OF BIOPROCESSES FOR THE TREATMENT OF WATER RESOURCES,salud,otro


Hay tres casos que fueron clasificados como `salud` por GPT 4, pero que nosotros clasificamos como `otro`. De todas formas, se puede notar que:
1. "THIOCOMPOSITES, A NOVEL DRUG CARRIER SYSTEM: PROOF-OF-CONCEPT FOR A POTENTIAL APPLICATION IN AQUACULTURE" habla de farmacología, pero aplicado a acuicultura, por lo cual se puede entender la confusión.
2. "TRANSMISSION DYNAMICS OF LEPTOSPIRA WITHIN-FARM: UNDERSTANDING AND MODELLING THE EFFECTS OF SOME HOSTS ON LEPTOSPIRA DYNAMICS AND MULTIPLE HOST-MULTIPLE STRAIN INTERACTIONS" menciona Leptospira, bacteria responsable de la leptospirosis, la cual es una enfermedad zoonótica y, como tal, puede
afectar a humanos. Esto también explica la confusión.

In [79]:
print(classification_report(df_sample["respuestas_salud"], df_sample["clasificación_normalizada_salud"]))

              precision    recall  f1-score   support

        otro       1.00      0.86      0.93        22
       salud       0.73      1.00      0.84         8

    accuracy                           0.90        30
   macro avg       0.86      0.93      0.88        30
weighted avg       0.93      0.90      0.90        30



#### Enfermedades específicas

Revisemos ahora los casos con discrepancias en la clasificación de enfermedades específicas

In [80]:
with pd.option_context('display.max_colwidth', 500):
  display(df_sample.loc[df_sample["clasificación_normalizada_específica"] != df_sample["respuestas_específicas"], ["glosa_específica", "clasificación_normalizada_específica", "respuestas_específicas"]])

,glosa_específica,clasificación_normalizada_específica,respuestas_específicas
16,"IRISINACTIVATOR: SUPLEMENTO PROTEICO, ACTIVADOR DE LA PRODUCCION DE IRISINA, PROMUEVE EL PARDEAMIENTO DEL TEJIDO ADIPOSO, FAVORECIENDO EL AUMENTO DE LA MASA MUSCULAR Y LA TERMOGENESIS.",diabetes,otro
23,TO BE BORN HIGH AND FLAT (ANDEAN ALTIPLANO): HEALING VASCULAR FUNCTION AND PULMONARY ARTERIAL HYPERTENSION BY A COMBINED TREATMENT IN NEONATAL LAMBS,cardiaca,otro


Acá se debe destacar que:
1. "IRISINACTIVATOR: SUPLEMENTO PROTEICO, ACTIVADOR DE LA
PRODUCCION DE IRISINA, PROMUEVE EL PARDEAMIENTO DEL TEJIDO
ADIPOSO, FAVORECIENDO EL AUMENTO DE LA MASA MUSCULAR Y
LA TERMOGENESIS". La irisina es una hormona que produce el tejido
muscular, y cuya relación con la diabetes está siendo estudiada.

2. "TO BE BORN HIGH AND FLAT (ANDEAN ALTIPLANO): HEALING
VASCULAR FUNCTION AND PULMONARY ARTERIAL HYPERTENSION
BY A COMBINED TREATMENT IN NEONATAL LAMBS". Si bien el título
del proyecto menciona corderos, también trata de función vascular
e hipertensión. Es importante recordar que, en el prompt para
generar esta clasificación, nunca se especificó que debía tratarse
de enfermedades humanas (esto sólo se exigió en la clasificación
salud-otro). Aun así, el proyecto efectivamente fue llevado a cabo
por personal del Instituto de Ciencias Biomédicas de la Facultad de Medicina de la Universidad de Chile, lo que da cuenta de la relación de este proyecto con la salud humana.

In [81]:
print(classification_report(df_sample["respuestas_específicas"], df_sample["clasificación_normalizada_específica"]))

              precision    recall  f1-score   support

    cardiaca       0.00      0.00      0.00         0
      cáncer       1.00      1.00      1.00         2
    diabetes       0.00      0.00      0.00         0
        otro       1.00      0.93      0.96        28

    accuracy                           0.93        30
   macro avg       0.50      0.48      0.49        30
weighted avg       1.00      0.93      0.97        30



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
